# LightGBM

In [ ]:
%load_ext autotime
%load_ext autoreload
%autoreload 2

In [ ]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import xgboost as xgb



from community_learning import base_model
from community_learning import features
from community_learning.example import add_region_train_test, region, get_two_region_data
from itertools import compress

time: 16.4 ms


In [ ]:
train_org, test = base_model.load_data()
product_dict = base_model.get_product_dict(train_org)
product_reverse_dict = base_model.get_product_reverse_dict(train_org)    




train, test = add_region_train_test(train_org, test, region)

train = base_model.encode_products(train_org)
#test_enc = features.calculate_target_as_one_column(test, features.feature_cols, features.target_cols)
#test_enc = base_model.encode_products(test)

train_X, train_y = base_model.x_y_split(train)

TypeError: Can only merge Series or DataFrame objects, a <class 'dict'> was passed

time: 2.13 s


In [ ]:
train_data = lgb.Dataset(train_X[base_model.feature_cols], label=train_y)

time: 23.3 ms


In [ ]:
param = {
    'objective': 'multiclass',
    'num_class': 22,
    'eta': 0.05,
    'min_child_weight': 1,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'max_depth': 8,
    'metric': 'multi_logloss',
    'verbosity': 2
}

time: 8.44 ms


In [ ]:
num_round = 50
bst = lgb.train(param, train_data, num_round)

KeyboardInterrupt: 

time: 8min 42s


In [ ]:
bst.best_score

defaultdict(collections.OrderedDict, {})

time: 18.8 ms


In [ ]:
preds = bst.predict(test[base_model.feature_cols])
preds.shape

(702435, 22)

time: 8.88 s


In [ ]:
#model = base_model.runXGB(train_X, train_y, base_model.feature_cols, use_gpu=True)
#xgtest = xgb.DMatrix(test[base_model.feature_cols])
#preds = model.predict(xgtest)

time: 44.2 s


In [ ]:
preds = np.argsort(preds, axis=1)
preds = np.fliplr(preds)[:,:7]
preds = pd.DataFrame(preds)
preds = preds.applymap(lambda x: product_reverse_dict[x])
preds['added_products'] = preds.apply(lambda x: list(x.values), axis=1)
preds = preds['added_products']

time: 32.9 s


In [ ]:
test_data = test.copy()
target_cols = base_model.target_cols
test_data.reset_index(inplace=True)
test_data['added_products'] = preds
test_data['truth_list'] = test_data[target_cols].apply(lambda x: list(compress(target_cols, x.values)), axis=1)
test_data['apk'] = test_data.apply(lambda x: base_model.apk(x['truth_list'], x['added_products']),axis=1)

time: 18.7 s


In [ ]:
print(f"mean average precision = {test_data['apk'].mean()}")

mean average precision = 0.022720981244268093
time: 10.1 ms
